In [ ]:
import pandas as pd
import geojson

In [ ]:
from geojson import Feature, Point, FeatureCollection

houses_df = pd.read_csv('../data/curated/property/property_4_lat_lon.csv')  
gym_df = pd.read_csv('../data/raw/external/victoria_gyms.csv')

houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
gym_df = gym_df.dropna(subset=['longitude', 'latitude'])


### create GeoJSON Feature 
def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'), 
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

# convert the data into Geojson
house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)
gym_features = gym_df.apply(lambda row: create_geojson_feature(row, 'gym'), axis=1).tolist()
gym_feature_collection = FeatureCollection(gym_features)

### save them
houses_geojson_output_path = '../data/raw/external/API/4/property_4_geojson.geojson'
gym_geojson_output_path = '../data/raw/external/API/4/victoria_gyms_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(gym_geojson_output_path, 'w') as f:
    geojson.dump(gym_feature_collection, f)


print("Geojson created")


In [ ]:
import geojson
import openrouteservice
import pandas as pd
import time

### api client
api_key = '5b3ce3597851110001cf6248c2b2602de2dc43afa9f186cdfc39caca' 
client = openrouteservice.Client(key=api_key)

### read Geojson
with open('../data/raw/external/API/4/property_4_geojson.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('../data/raw/external/API/4/victoria_gyms_geojson.geojson', 'r') as f:
    gym_geojson = geojson.load(f)

### create the function to get the latitude and longtitude
def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
gym_coordinates = extract_coordinates(gym_geojson)


house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
gym_coords = [(lon, lat) for lat, lon, _ in gym_coordinates]

### using api to calculate the driving distance
def calculate_distance_matrix_in_batches(house_coords, gym_coords, batch_size=7):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + gym_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(gym_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
             ### find mini distance
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
  
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_gym_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_gym = gym_coordinates[closest_gym_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_gym': closest_gym,
                        'minimum_distance_gym': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)  
    
    return all_shortest_distances

### batch size the data
shortest_distances = calculate_distance_matrix_in_batches(house_coords, gym_coords, batch_size=7)

### save data
shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '../data/curated/external/API/4/closest_gym_distance_final_4.csv'
shortest_distances_df.to_csv(output_path, index=False)

